<a href="https://colab.research.google.com/github/mchivuku/csb659-project/blob/master/Disentangling_by_Factorising.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# Disentagling by Factorizing:
Implementation for paper - https://arxiv.org/pdf/1802.05983.pdf



In [0]:
%%capture
!pip install tqdm six


!pip install bokeh
!pip install tensorboard
!pip install livelossplot

!pip install tensorboard

!pip install tensorboardX



## Connect to google drive

In [3]:
from google.colab import drive

drive.mount("/content/drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
%cd drive/My\ Drive/Masters-DS/CSCI-B659/project/examples/disentangling-fac

## Import Libraries

In [4]:
from glob import glob
import os
import numpy as np
import matplotlib.pyplot as plt
import shutil
from torchvision import transforms
from torchvision import models
import torchvision
import torch
from torch.autograd import Variable
import torch.nn as nn
from torch.optim import lr_scheduler
from torch import optim
from torchvision.datasets import ImageFolder
from torchvision.utils import make_grid
import time
from torchvision import datasets,transforms
import torch.nn.functional as F
import torch.optim as optim


import os

## Plotting library

from bokeh.plotting import figure
from bokeh.io import show
from bokeh.models import LinearAxis, Range1d
from livelossplot import PlotLosses


plt.style.use('tableau-colorblind10')

print('Torch', torch.__version__, 'CUDA', torch.version.cuda)
print('Device:', torch.device('cuda:0'))
print(torch.cuda.is_available())

is_cuda = torch.cuda.is_available()
device = torch.device ( "cuda:0" if torch.cuda.is_available () else "cpu" )

Torch 1.0.1.post2 CUDA 10.0.130
Device: cuda:0
True


In [0]:
torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True

init_seed = 1
torch.manual_seed(init_seed)
torch.cuda.manual_seed(init_seed)
np.random.seed(init_seed)


## Class Params

In [0]:
import os

class Params:
  max_iter=1e6
  batch_size = 64
  z_dim = 20
  gamma = 6.4
  lr_vae = 1e-4
  beta1_vae = 0.9
  beta2_vae = 0.999
  lr_D = 1e-4
  beta1_D = 0.5
  beta2_D  = 0.9
  image_size = 784
  viz_on = True
  print_iter = 500
  num_workers = 2
  output_dir = "outputs"
  ckpt_save_iter = 100
  
  
  
def mkdirs(path):
    if not os.path.exists(path):
        os.makedirs(path)

mkdirs("logs")

In [89]:
LOG_DIR = './logs/'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)


! wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 6006 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

--2019-03-25 14:51:36--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.203.66.95, 52.200.123.104, 52.204.188.97, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.203.66.95|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13584026 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.1’

ngrok-stable-linux- 100%[===================>]  12.95M  12.7MB/s    in 1.0s    

2019-03-25 14:51:37 (12.7 MB/s) - ‘ngrok-stable-linux-amd64.zip.1’ saved [13584026/13584026]

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ngrok                   
https://adbd0abf.ngrok.io


In [0]:
from tensorboardX import SummaryWriter
writer = SummaryWriter()

## Models

In [0]:
import torch.nn as nn
import torch.nn.init as init

In [0]:
class Discriminator(nn.Module):
    def __init__(self,z_dim):
        super(Discriminator, self).__init__()
        self.z_dim= z_dim
        self.net = nn.Sequential(
            nn.Linear(z_dim, 1000),
            nn.LeakyReLU(0.2, True),
            nn.Linear(1000, 1000),
            nn.LeakyReLU(0.2, True),
            nn.Linear(1000, 1000),
            nn.LeakyReLU(0.2, True),
            nn.Linear(1000, 1000),
            nn.LeakyReLU(0.2, True),
            nn.Linear(1000, 1000),
            nn.LeakyReLU(0.2, True),
            nn.Linear(1000, 2),
        )
        
        self.weight_init()


    def weight_init(self, mode='normal'):
        if mode == 'kaiming':
            initializer = kaiming_init
        elif mode == 'normal':
            initializer = normal_init

        for block in self._modules:
            for m in self._modules[block]:
                initializer(m)

    def forward(self, z):
        return self.net(z).squeeze()


In [0]:
## FactorVAE, Encoder and Decoder Architecture
class FactorVAE(nn.Module):
  """
  Encoder and Decoder architecture
  """
  def __init__(self,z_dim = 20):
    super(FactorVAE,self).__init__()
    self.z_dim= z_dim
    
    self.mu_ = nn.Sequential(
        #28x28->12x12
        nn.Conv2d(1,8,5,2,0,bias=False),
        nn.BatchNorm2d(8),
        nn.ReLU(True),
        #12x12->4x4
        nn.Conv2d(8,64,5,2,0,bias=False),
        nn.BatchNorm2d(64),
        nn.ReLU(True),
        #4x4->1x1: 20,1,1
        nn.Conv2d(64,20,4,1,0,bias=False),
        nn.ReLU(True)
        )


    self.logsigma_ = nn.Sequential(
        #28x28->12x12
        nn.Conv2d(1,8,5,2,0,bias=False),
        nn.BatchNorm2d(8),
        nn.ReLU(True),
        #12x12->4x4
        nn.Conv2d(8,64,5,2,0,bias=False),
        nn.BatchNorm2d(64),
        nn.ReLU(True),
        #4x4->1x1: 20,1,1
        nn.Conv2d(64,20,4,1,0,bias=False),
        nn.ReLU(True)
        )


    self.decode = nn.Sequential(
        #1x1->4x4
        nn.ConvTranspose2d(20,20*8,4,1,0,bias=False),  #(ic,oc,kernel,stride,padding)
        nn.BatchNorm2d(20*8), 
        nn.ReLU(True),
        nn.ConvTranspose2d(20*8,20*16,4,2,1,bias=False), #4x4->8x8
        nn.BatchNorm2d(20*16),
        nn.ReLU(True),
        nn.ConvTranspose2d(20*16,20*32,4,2,1,bias=False), #8x8->16x16
        nn.BatchNorm2d(20*32),
        nn.ReLU(True),
        nn.ConvTranspose2d(20*32,1,2,2,2,bias=False), #16x16->28x28
        nn.Sigmoid()
        )
      
    self.weight_init()
    
    
  def weight_init(self, mode='normal'):
        if mode == 'kaiming':
            initializer = kaiming_init
        elif mode == 'normal':
            initializer = normal_init

        for block in self._modules:
            for m in self._modules[block]:
                initializer(m)
    
  def reparametrize(self, mu, logvar):
        std = logvar.mul(0.5).exp_()
        eps = std.data.new(std.size()).normal_()
        return eps.mul(std).add_(mu)
      
      
  def encode_new(self,x):
        return self.mu_(x), self.logsigma_(x)

      
  def forward(self, x, no_dec=False):
        
        mu, logvar = self.encode_new(x)
        
        z = self.reparametrize(mu, logvar)

        if no_dec:
            return z.squeeze()
        else:
            x_recon = self.decode(z).view(x.size())
            return x_recon, mu, logvar, z.squeeze()
          
          
          
          
          
## Weight initializations:
def kaiming_init(m):
    if isinstance(m, (nn.Linear, nn.Conv2d)):
        init.kaiming_normal_(m.weight)
        if m.bias is not None:
            m.bias.data.fill_(0)
    elif isinstance(m, (nn.BatchNorm1d, nn.BatchNorm2d)):
        m.weight.data.fill_(1)
        if m.bias is not None:
            m.bias.data.fill_(0)


def normal_init(m):
    if isinstance(m, (nn.Linear, nn.Conv2d)):
        init.normal_(m.weight, 0, 0.02)
        if m.bias is not None:
            m.bias.data.fill_(0)
    elif isinstance(m, (nn.BatchNorm1d, nn.BatchNorm2d)):
        m.weight.data.fill_(1)
        if m.bias is not None:
            m.bias.data.fill_(0)

In [103]:
## initialize the model
VAE = FactorVAE(Params.z_dim).to(device)
D = Discriminator(Params.z_dim).to(device)
print(VAE)
print()
print(D)

FactorVAE(
  (mu_): Sequential(
    (0): Conv2d(1, 8, kernel_size=(5, 5), stride=(2, 2), bias=False)
    (1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): Conv2d(8, 64, kernel_size=(5, 5), stride=(2, 2), bias=False)
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace)
    (6): Conv2d(64, 20, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (7): ReLU(inplace)
  )
  (logsigma_): Sequential(
    (0): Conv2d(1, 8, kernel_size=(5, 5), stride=(2, 2), bias=False)
    (1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): Conv2d(8, 64, kernel_size=(5, 5), stride=(2, 2), bias=False)
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace)
    (6): Conv2d(64, 20, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (7): ReLU(inplace)
  )
  (decode): Sequentia

In [0]:
optim_VAE = optim.Adam(VAE.parameters(), lr=Params.lr_vae,betas=(Params.beta1_vae, Params.beta2_vae))

In [0]:
optim_D = optim.Adam(D.parameters(), lr=Params.lr_D,
                                  betas=(Params.beta1_D, Params.beta2_D))

In [0]:
nets = [VAE, D]

## Set model to train

def net_mode(nets, train):
   if not isinstance(train, bool):
      raise ValueError("Only bool type allowed")
   for net in nets:
    if train:
      net.train()
    else:
      net.eval()


## Train

In [107]:
## Save and load checkpoints
from tqdm import tqdm

ckpt_dir = "./logs"
pbar = tqdm(total=Params.max_iter)


"""
Save checkpoint - model params, optimizer params, states etc.
"""
def save_checkpoint(global_iter,ckptname='last', verbose=True):
        model_states = {'D': D.state_dict(),
                        'VAE': VAE.state_dict()}
    
        optim_states = {'optim_D':optim_D.state_dict(),
                        'optim_VAE':optim_VAE.state_dict()}
      
        states = {'iter':global_iter,
                  'model_states':model_states,
                  'optim_states':optim_states}

        filepath = os.path.join(ckpt_dir, str(ckptname))
        with open(filepath, 'wb+') as f:
            torch.save(states, f)
        if verbose:
            pbar.write("=> saved checkpoint '{}' (iter {})".format(filepath, global_iter))

"""
Load checkoint
"""           
def load_checkpoint(global_iter,ckptname='last', verbose=True):
    if ckptname == 'last':
        ckpts = os.listdir(ckpt_dir)
        if not ckpts:
            if verbose:
                pbar.write("=> no checkpoint found")
            return

        ckpts = [int(ckpt) for ckpt in ckpts]
        ckpts.sort(reverse=True)
        ckptname = str(ckpts[0])

    filepath = os.path.join(ckpt_dir, ckptname)
    if os.path.isfile(filepath):
        with open(filepath, 'rb') as f:
            checkpoint = torch.load(f)

        global_iter = checkpoint['iter']
        VAE.load_state_dict(checkpoint['model_states']['VAE'])
        D.load_state_dict(checkpoint['model_states']['D'])
        optim_VAE.load_state_dict(checkpoint['optim_states']['optim_VAE'])
        optim_D.load_state_dict(checkpoint['optim_states']['optim_D'])
        pbar.update(global_iter)
        
        
        if verbose:
            pbar.write("=> loaded checkpoint '{} (iter {})'".format(filepath, global_iter))
    else:
        if verbose:
            pbar.write("=> no checkpoint found at '{}'".format(filepath))




  0%|          | 0/1000000.0 [00:00<?, ?it/s]

## Dataset Loader

In [0]:
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import ImageFolder
from torchvision import transforms
import random
class CustomTensorDataset(Dataset):
    def __init__(self, data_tensor, transform=None):
        self.data_tensor = data_tensor
        self.transform = transform
        self.indices = range(len(self))

    def __getitem__(self, index1):
        index2 = random.choice(self.indices)

        img1 = self.data_tensor[index1]
        img2 = self.data_tensor[index2]
        if self.transform is not None:
            img1 = self.transform(img1)
            img2 = self.transform(img2)

        return img1, img2

    def __len__(self):
        return self.data_tensor.size(0)

class CustomImageFolder(ImageFolder):
    def __init__(self, root, transform=None):
        super(CustomImageFolder, self).__init__(root, transform)
        self.indices = range(len(self))

    def __getitem__(self, index1):
        index2 = random.choice(self.indices)

        path1 = self.imgs[index1][0]
        path2 = self.imgs[index2][0]
        img1 = self.loader(path1)
        img2 = self.loader(path2)
        if self.transform is not None:
            img1 = self.transform(img1)
            img2 = self.transform(img2)

        return img1, img2
      
dset_dir = "../vae/MNIST/data/MNIST/processed/training.pt"
data, targets = torch.load(dset_dir)
train_kwargs = {'data_tensor':data}
dset = CustomTensorDataset


train_data = dset(**train_kwargs)
train_loader = DataLoader(train_data,
          batch_size=Params.batch_size,
        shuffle=True,
        num_workers=Params.num_workers,
        pin_memory=True,
 drop_last=True)



## Losses

1. reconstruction loss - binary cross entropy
2. KL divergence 
3. permute dimz


In [0]:
def recon_loss(x, x_recon):
    n = x.size(0)
    loss = F.binary_cross_entropy_with_logits(x_recon, x, size_average=False).div(n)
    return loss


def kl_divergence(mu, logvar):
    kld = -0.5*(1+logvar-mu**2-logvar.exp()).sum(1).mean()
    return kld


def permute_dims(z):
    assert z.dim() == 2

    B, _ = z.size()
    perm_z = []
    for z_j in z.split(1, 1):
        perm = torch.randperm(B).to(z.device)
        perm_z_j = z_j[perm]
        perm_z.append(perm_z_j)

    return torch.cat(perm_z, 1)

In [0]:
net_mode(nets,True)


In [0]:
## real outputs
ones = torch.ones(Params.batch_size, dtype=torch.long, device=device)
# fake outputs
zeros = torch.zeros(Params.batch_size, dtype=torch.long, device=device)


In [0]:
out = False
global_iter = 0
while not out:
  for x_true1, x_true2 in train_loader:
    global_iter +=1
    pbar.update(1)
    
    
    x_true1 = x_true1.to(device)
    
    x_recon, mu,logvar, z = VAE(x_true1)
    vae_recon_loss  = recon_loss(x_true1, x_recon)
    vae_kld  = kl_divergence(mu,logvar)
    
    ## discriminator 
    D_z = D(z)
    # total correlation
    vae_tc_loss = (D_z[:,:1] - D_z[:,1:]).mean()
    
    vae_loss = vae_recon_loss + vae_kld +\
    Params.gamma*vae_tc_loss
    
    optim_VAE.zero_grad()
    vae_loss.backward(retain_graph= True)
    
    optim_VAE.step()
    
    ## Labels
    x_true2 = x_true2.to(device)
    z_prime = VAE(x_true2, no_dec=True)
    
    z_pperm = permute_dims(z_prime).detach()
    
    D_z_pperm = D(z_pperm)
    
    D_tc_loss = 0.5*(F.cross_entropy(D_z, zeros) + F.cross_entropy(D_z_pperm, ones))
    
    optim_D.zero_grad()
    
    D_tc_loss.backward()
    
    optim_D.step()
    
    if global_iter%Param.print_iter == 0:
      pbar.write('[{}] vae_recon_loss:{:.3f} vae_kld:{:.3f} vae_tc_loss:{:.3f} D_tc_loss:{:.3f}'.format(global_iter, vae_recon_loss.item(), vae_kld.item(), vae_tc_loss.item(), D_tc_loss.item()))
      
      writer.add_scalars('data/scalar_group', {'vae_loss': vae_loss.item(),
                                              'D_tc_loss': D_tc_loss.item()}, global_iter)
      
      x = utils.make_grid(F.sigmoid(x_recon).data.cpu(), 
                           normalize=True, scale_each=True)
      
      writer.add_image('Image', x, global_iter)
      
      
    if  global_iter >= Params.max_iter:
      out = True
      break
          
          

          
        
      
    if global_iter%ckpt_save_iter == 0:
      save_checkpoint(global_iter)
      
      
    
      
      
    